<a href="https://colab.research.google.com/github/JuliKM/InvertirOnLine/blob/master/Julieta_Kogan_Tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo práctico 2

### Obtener datasets

In [ ]:
pip install kaggle

In [ ]:
!gdown kaggle competitions download -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--id] url_or_id
gdown: error: unrecognized arguments: competitions download -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2


In [ ]:
!kaggle competitions download -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip tp-n2-aprendizaje-profundo-2021-by-datitos-v2.zip

unzip:  cannot find or open tp-n2-aprendizaje-profundo-2021-by-datitos-v2.zip, tp-n2-aprendizaje-profundo-2021-by-datitos-v2.zip.zip or tp-n2-aprendizaje-profundo-2021-by-datitos-v2.zip.ZIP.


### Cargar datasets

In [19]:
import pandas as pd
import numpy as np

df = pd.read_csv('fifa2021_training.csv')
df_infer = pd.read_csv('fifa2021_test.csv')

### Breve análisis exploratorio

In [4]:
df.iloc[0]

ID                            243620
Name                   Adam Hellborg
Natinality                    Sweden
Overal                            64
Potential                         73
Height                           188
Weight                            79
PreferredFoot                      R
BirthDate              July 30, 1998
Age                               22
PlayerWorkRate         Medium/Medium
WeakFoot                           3
SkillMoves                         2
Value                        1.2e+06
Wage                            1500
Club                       IK Sirius
Club_KitNumber                     2
Club_JoinedClub         Jan. 8, 2020
Club_ContractLength             2022
BallControl                       62
Dribbling                         55
Marking                           60
SlideTackle                       57
StandTackle                       60
Aggression                        71
Reactions                         58
Interceptions                     60
V

### Particionar datasets

In [2]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, stratify=df.Position, train_size=0.9, random_state=42)

### Definir transformaciones

In [23]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

variables_descartar = [
   'Position', # variable objetivo
    'ID',
    'Name',
    'Natinality',
    'Potential',
    'BirthDate',
    'WeakFoot',
    'Value',
    'Wage',
    'Club',
    'Club_KitNumber',
    'Club_JoinedClub',
    'Club_ContractLength',
]

variables_categóricas = df.drop(columns=variables_descartar).select_dtypes(include=np.object).columns
variables_numéricas   = df.drop(columns=variables_descartar).select_dtypes(include=np.number).columns

transformador = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'),  variables_categóricas), # PreferredFoot, PlayerWorkRate, Sex resolver nacionalidad con handle_unknown='ignore' Eso trae otro probl despues, asi q por ahora vamos con eliminar los dos paises q me joden
    (StandardScaler(), variables_numéricas),   # Overal, Potential, Height, etc.
    remainder='drop' # descarta las columnas no mencionadas en las transformaciones
)

### Entrenar transformador

Esencialmente, calculamos los **promedios** y las **desviaciones estándares** que usaremos para la estandarización.

**ESTOS VALORES SOLO DEBEN SER OBTENIDOS DEL DATASET DE ENTRENAMIENTO**.

In [24]:
transformador.fit(df_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('onehotencoder',
                                 OneHotEncoder(categories='auto', drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='ignore',
                                               sparse=True),
                                 Index(['PreferredFoot', 'PlayerWorkRate', 'Sex'], dtype='object')),
                                ('standardscaler',
                                 StandardScaler(copy=True, with_mean=T...
       'Dribbling', 'Marking', 'SlideTackle', 'StandTackle', 'Aggression',
       'Reactions', 'Interceptions', 'Vision', 'Composure', 'Crossing',
       'ShortPass', 'LongPass', 'Acceleration', 'Stamina', 'Strength',
       'Balance', 'SprintSpeed', 'Agility', 'Jumping', 'Heading', 'ShotPower',
       'Finishing', 'Lon

### Transformar datasets

In [25]:
# el transformador se queja si falta alguna columna de df_train :/  
df_infer['Position'] = None


X_train = transformador.transform(df_train)
X_valid = transformador.transform(df_valid)
X_infer = transformador.transform(df_infer)

### Transformar variable objetivo

Como la variable objetivo es del tipo string, hay que llevarla a un tipo numérico para que PyTorch pueda procesarla.

Este transformador mapea posiciones DEF, FWD, GK, MID a enteros **y viceversa** — la transformación inversa será útil para convertir las predicciones (enteros) en posiciones otra vez.

In [26]:
from sklearn.preprocessing import LabelEncoder

transformador_etiquetas = LabelEncoder()

transformador_etiquetas.fit(df_train.Position)

y_train = transformador_etiquetas.transform(df_train.Position)
y_valid = transformador_etiquetas.transform(df_valid.Position)

### Instanciar Datasets de PyTorch

El aprendizaje profundo es especialmente efectivo para imágenes y texto; para datos tabulares (como un DataFrame) el aprendizaje de máquinas clásico suele funcionar bastante bien, de ahí que PyTorch no cuente con facilidades para tratar este tipo de problemas.

In [27]:
from torch.utils.data import Dataset

class Tabular(Dataset):
    def __init__(self, X, y=None):
        self.X = X.astype(np.float32) # soluciona "Expected object of scalar type Float but got scalar type Double"
        self.y = y 

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, item):
        if self.y is None:
            return self.X[item]
        else:
            return self.X[item], self.y[item]

        
ds_train = Tabular(X_train, y_train)

In [28]:
ds_train[10]

(array([ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  2.2005258 ,  1.6827489 ,
         0.69650096,  1.9552506 , -0.49693993,  1.5163243 ,  1.0263261 ,
         0.5255314 , -1.4219726 , -0.93868953,  0.10657337,  2.1206129 ,
         0.13522774,  1.5071335 ,  2.1563685 ,  0.6801946 ,  1.1708272 ,
         1.3763508 , -0.25280562,  0.7491539 ,  1.286718  , -2.0082333 ,
        -0.4727765 , -0.531109  , -0.06149001,  1.7349557 ,  2.030825  ,
         2.0725315 ,  1.5956575 ,  0.90770805,  0.6198386 ,  1.1499894 ,
         1.857724  , -0.42461264, -0.31164196, -0.19319764, -0.09562566],
       dtype=float32), 1)

### Instanciar DataLoaders de PyTorch

In [29]:
from torch.utils.data import DataLoader

dl_train = DataLoader(ds_train, batch_size=32, shuffle=False)

In [30]:
import torch
X_valid = torch.tensor(X_valid).float()
X_infer = torch.tensor(X_infer).float()
y_valid = torch.tensor(y_valid)

### Instanciar modelo

In [31]:
from sklearn.model_selection import train_test_split 

In [33]:
import torch
import torch.nn as nn

In [37]:
IN  = X_train.shape[1]
OUT = len(transformador_etiquetas.classes_)

modelo = nn.Sequential(
    nn.Linear(IN,  8),
    nn.Linear( 8, 160), nn.ReLU(),
    nn.Linear(160, 32), nn.ReLU(), 
    nn.Linear(32, OUT)
)

In [38]:
criterio = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(modelo.parameters(), lr=0.0001)

### Entrenar modelo

In [39]:
from sklearn.metrics import balanced_accuracy_score

ÉPOCAS = 10

for época in range(ÉPOCAS):
    # activa capas Dropout, BatchNorm si las hubiese
    modelo.train()

    pérdidas_train = []
    
    for X_lote, y_lote in dl_train:
        optimizador.zero_grad()

        predicciones = modelo(X_lote)
        pérdida = criterio(predicciones, y_lote)

        pérdida.backward()
        optimizador.step()
        
        pérdidas_train.append(pérdida.item())
    
    # desactiva capas Dropout, BatchNorm si las hubiese
    modelo.eval()
    
    with torch.no_grad():
        predicciones = modelo(X_valid)
        pérdida = criterio(predicciones, y_valid)
        
        y_pred = predicciones.argmax(dim=1) # selecciona la clase con mayor probabilidad
        
        efectividad = balanced_accuracy_score(y_valid, y_pred)
    
    
    print(f'{época:3d}  |  Train loss: {np.mean(pérdidas_train):.3f}    Valid loss: {pérdida:.3f}    Valid accuracy: {efectividad:.2f}')

  0  |  Train loss: 1.004    Valid loss: 0.681    Valid accuracy: 0.70
  1  |  Train loss: 0.526    Valid loss: 0.427    Valid accuracy: 0.84
  2  |  Train loss: 0.368    Valid loss: 0.353    Valid accuracy: 0.87
  3  |  Train loss: 0.315    Valid loss: 0.320    Valid accuracy: 0.88
  4  |  Train loss: 0.291    Valid loss: 0.304    Valid accuracy: 0.88
  5  |  Train loss: 0.279    Valid loss: 0.294    Valid accuracy: 0.89
  6  |  Train loss: 0.271    Valid loss: 0.287    Valid accuracy: 0.88
  7  |  Train loss: 0.266    Valid loss: 0.282    Valid accuracy: 0.89
  8  |  Train loss: 0.262    Valid loss: 0.278    Valid accuracy: 0.89
  9  |  Train loss: 0.259    Valid loss: 0.274    Valid accuracy: 0.89


### Inferir datos de prueba

In [42]:
with torch.no_grad():
    y_infer = modelo(X_infer).argmax(dim=1)

df_infer['Position'] = transformador_etiquetas.inverse_transform(y_infer)

(
    df_infer[['ID', 'Position']]
    .rename(columns={'ID':'Id', 'Position':'Category'})
    .to_csv('submit.csv', index=False)
)

### Subir predicciones

In [107]:
!kaggle competitions submit -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2 -f submit.csv -m "Brasil, decime qué se siente"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
